In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Directories
base_dir = '/kaggle/input/tuberculosisdata/TB_Data'
train_dir = os.path.join(base_dir, 'Train')
val_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Test')

# Datasets & Dataloaders
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transforms)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes
print("✅ Classes:", class_names)


In [ ]:
# Load pretrained ViT
vit_model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)

# Modify classifier for binary classification
in_features = vit_model.heads[0].in_features
vit_model.heads = nn.Sequential(nn.Linear(in_features, 2))
vit_model = vit_model.to(device)


In [ ]:
import os

# Base dataset path
base_path = "/kaggle/input/tuberculosisdata/TB_Data"

# Function to count images in each subfolder
def count_images(folder_path):
    counts = {}
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if os.path.isdir(class_path):
            num_images = len(os.listdir(class_path))
            counts[class_name] = num_images
    return counts

# Count for Train, Validation, Test
train_counts = count_images(os.path.join(base_path, "Train"))
val_counts = count_images(os.path.join(base_path, "Validation"))
test_counts = count_images(os.path.join(base_path, "Test"))

# Print the results
print("📊 Train Set Image Counts:", train_counts)
print("📊 Validation Set Image Counts:", val_counts)
print("📊 Test Set Image Counts:", test_counts)


In [ ]:
# Compute class weights
class_counts = [len(os.listdir(os.path.join(train_dir, c))) for c in class_names]
weights = 1. / torch.tensor(class_counts, dtype=torch.float)
weighted_loss = nn.CrossEntropyLoss(weight=weights.to(device))

# Optimizer
optimizer = torch.optim.Adam(vit_model.parameters(), lr=1e-4)


In [ ]:
def evaluate_model(model, loader, criterion):
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return val_loss / len(loader), correct / total

# Store metrics for plotting
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=20):
    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        avg_train_loss = running_loss / len(train_loader)
        train_acc = correct / total
        val_loss, val_acc = evaluate_model(model, val_loader, criterion)

        train_losses.append(avg_train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        print(f"📉 Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"📈 Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        # Save best model (optional, still kept)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")


In [ ]:
train_model(vit_model, weighted_loss, optimizer, train_loader, val_loader, epochs=50)


In [ ]:
# Load best model
vit_model.load_state_dict(torch.load("best_model.pth"))
vit_model.eval()

all_preds, all_labels, all_probs = [], [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = vit_model(images)
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(cm, display_labels=class_names)
disp.plot(cmap='Blues', values_format='d')
plt.title("🧪 Confusion Matrix on Test Set")
plt.show()

# Classification Report
print("📋 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

# ROC Curve
fpr, tpr, _ = roc_curve(all_labels, all_probs)
roc_auc = roc_auc_score(all_labels, all_probs)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Vision Transformer")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Accuracy Curve
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.grid()
plt.show()

# Loss Curve
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()
plt.grid()
plt.show()


In [ ]:
print("📊 Accuracy per Epoch:")
print(f"{'Epoch':<6} {'Train Accuracy (%)':<20} {'Val Accuracy (%)':<20}")
print("-" * 50)

for epoch, (train_acc, val_acc) in enumerate(zip(train_accuracies, val_accuracies), start=1):
    print(f"{epoch:<6} {train_acc * 100:<20.2f} {val_acc * 100:<20.2f}")


In [ ]:
from sklearn.metrics import accuracy_score

def get_final_accuracy(model, loader, set_name="Set"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"✅ Final Accuracy on {set_name}: {acc * 100:.2f}%")
    return acc * 100

# Make sure your best model is loaded
vit_model.load_state_dict(torch.load("best_model.pth"))

# Calculate accuracies
final_train_acc = get_final_accuracy(vit_model, train_loader, "Train Set")
final_val_acc = get_final_accuracy(vit_model, val_loader, "Validation Set")
final_test_acc = get_final_accuracy(vit_model, test_loader, "Test Set")


In [ ]:
from PIL import Image

# Path to the uploaded image
image_path = '/kaggle/input/tuberculosisdata/TB_Data/Test/Normal/Normal-1009.png'  # change this if needed

# Load and preprocess the image
image = Image.open(image_path).convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

input_tensor = transform(image).unsqueeze(0).to(device)

# Load your trained model
vit_model.eval()
with torch.no_grad():
    output = vit_model(input_tensor)
    predicted_class = torch.argmax(output, 1).item()

# Class names (ensure it matches your training dataset)
class_names = ['Normal', 'Tuberculosis']
print(f"✅ Predicted Class: {class_names[predicted_class]}")


In [ ]:
plt.imshow(image)
plt.axis('off')
plt.title(f"Prediction: {class_names[predicted_class]}")
plt.show()


In [ ]:
from PIL import Image

# Path to the uploaded image
image_path = '/kaggle/input/tuberculosisdata/TB_Data/Test/Tuberculosis/Tuberculosis-113_aug0.png'  # change this if needed

# Load and preprocess the image
image = Image.open(image_path).convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

input_tensor = transform(image).unsqueeze(0).to(device)

# Load your trained model
vit_model.eval()
with torch.no_grad():
    output = vit_model(input_tensor)
    predicted_class = torch.argmax(output, 1).item()

# Class names (ensure it matches your training dataset)
class_names = ['Normal', 'Tuberculosis']
print(f"✅ Predicted Class: {class_names[predicted_class]}")


In [ ]:
plt.imshow(image)
plt.axis('off')
plt.title(f"Prediction: {class_names[predicted_class]}")
plt.show()


In [ ]:
# Save the entire model (architecture + weights)
torch.save(vit_model, "vit_full_model.pth")
print("✅ ViT model saved as vit_full_model.pth")
